In [48]:
from keras import backend as K
import os

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("theano")

Using Theano backend.


In [63]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ProgbarLogger
import numpy as np


In [57]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 50 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

l2_lambda = 0.0001

In [58]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

num_train, depth, height, width = X_train.shape # there are 50000 training examples in CIFAR-10 
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_train) # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels



In [59]:
#X_val = X_train[45000:]
#Y_val = Y_train[45000:]

#X_train = X_train[:45000]
#Y_train = Y_train[:45000]

In [60]:
# Setup data generator
datagen = ImageDataGenerator(
            width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1, # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,
            vertical_flip=True
)
datagen.fit(X_train)


In [61]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras!

inp_norm = BatchNormalization(axis=1)(inp)

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), 
                       padding='same', 
                       kernel_regularizer=l2(l2_lambda), 
                       kernel_initializer='he_uniform',
                       activation='relu')(inp_norm)
conv_1_norm = BatchNormalization(axis=1)(conv_1)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), 
                       padding='same', 
                       kernel_regularizer=l2(l2_lambda), 
                       kernel_initializer='he_uniform',
                       activation='relu')(conv_1_norm)
conv_2_norm = BatchNormalization(axis=1)(conv_2)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2_norm)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, 
                       (kernel_size, kernel_size), 
                       padding='same', 
                       kernel_regularizer=l2(l2_lambda), 
                       kernel_initializer='he_uniform', 
                       activation='relu')(drop_1)
conv_3_norm = BatchNormalization(axis=1)(conv_3)
conv_4 = Convolution2D(conv_depth_2, 
                      (kernel_size, kernel_size), 
                       padding='same', 
                       kernel_regularizer=l2(l2_lambda), 
                       kernel_initializer='he_uniform',
                       activation='relu')(conv_3_norm)
conv_4_norm = BatchNormalization(axis=1)(conv_4)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4_norm)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
hidden_norm = BatchNormalization(axis = 1)(hidden)
drop_3 = Dropout(drop_prob_2)(hidden_norm)
out = Dense(num_classes, 
            kernel_initializer='glorot_uniform', 
            kernel_regularizer=l2(l2_lambda),
            activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

In [65]:
#model.fit_generator(datagen.flow(X_train, Y_train,
#                        batch_size=batch_size),
#                        samples_per_epoch=X_train.shape[0],
#                        nb_epoch=num_epochs,
#                        validation_data=(X_val, Y_val),
#                        verbose=1)

model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, 
          nb_epoch=num_epochs,
          verbose=1, 
          validation_split=0.1,
          callbacks=[
              EarlyStopping(monitor='val_loss', patience=5), 
              ProgbarLogger()
          ]
         )

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 45000 samples, validate on 5000 samples
Epoch 1/50
Epoch 1/50
 5952/45000 [==>...........................] - ETA: 644s - loss: 2.4265 - acc: 0.2972

KeyboardInterrupt: 

In [47]:
model.evaluate(X_train, Y_train, verbose=1) # Evaluate the trained model on the test set!

49952/50000 [============================>.] - ETA: 0s

[0.28307561039447782, 0.94608000000000003]